In [1]:
from tkinter import *
from tkinter.filedialog import askopenfile
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
import pptk 
import matplotlib.pyplot as plt
import laspy as lp
import time
import requests
import json
from pypcd import pypcd as pc

In [2]:
x = requests.get('http://localhost:5000/getdata')

data  = np.asarray(json.loads(x.content)['data'])
print(data)

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /getdata (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002B4CA590F48>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [2]:
class PPTKPlotter:
    def plot3D(self,data,colorAttr,pointSize):
        #attr1 = pptk.rand(100)
        data = np.asarray(data)
        v = pptk.viewer(data)
        v.attributes(colorAttr)
        if pointSize != None:
            v.set(point_size=pointSize)
       
        
        
        
    def plotStream3D(self,API):
        '''v.capture('screenshot.png')
        time.sleep(10)
        print('reached here')
        v.clear()
        time.sleep(10)
        print('reached here too')
        v.load(data)'''
        self.filecount = 1
        x = requests.get(API)
        data  = np.asarray(json.loads(x.content)['data'])
        v = pptk.viewer(data)
        v.attributes(data[:,0])
        #v.set(point_size=0.05)
        
        while(True):
            time.sleep(7)
            x = requests.get(API)
            data  = np.asarray(json.loads(x.content)['data'])
            v.load(data)
            v.attributes(data[:,0])
            #v.set(point_size=0.05)
            #v.set(lookat=[5,5,5])
            #pd.DataFrame(data).to_csv("file"+str(self.filecount)+".csv")
            self.filecount +=1
            
        
        
        

In [3]:
class PCDReader:
    def __init__(self,fileName):
        self.pointCloud = pc.PointCloud.from_path(fileName)
        self.dataFrame  = pd.DataFrame(self.pointCloud.pc_data)
        self.m = None
        self.n = None
        global X
        global Y
        global Z
        self.X= 0
        self.Y =1
        self.Z =2
        self.colorAttr = 2
        self.varX = IntVar()
        self.varY = IntVar()
        self.varZ = IntVar()
        self.varColor = IntVar()
        self.pointSize = None
    def onXRadioButtonSelect(self):
        self.X = self.varX.get()
        print(self.X)
        
    def onYRadioButtonSelect(self):
        self.Y = self.varY.get()
        print(self.Y)
        
    def onZRadioButtonSelect(self):
        self.Z = self.varZ.get()
    def onColorRadioButtonSelect(self):
        self.colorAttr = self.varColor.get()
        
    def invokePPTKPlotter(self):
        pptkInstance = PPTKPlotter()
        
        self.data  = self.dataFrame.iloc[:,[self.X,self.Y,self.Z]]
        if self.pointSizeField.get() != '':
            self.pointSize = float(self.pointSizeField.get())
        else:
            self.pointSize = None
        
        pptkInstance.plot3D(self.data,self.dataFrame.iloc[:,self.colorAttr],self.pointSize)
    def loadVisualizeMenuOptions(self,_labelFrame):
        
        self.m+=1
        self.n=1
        labelX = Label(_labelFrame, text ='X-Axis:').grid(row=self.m,column=self.n)
        
        for i in range(0,self.dataFrame.columns.size):
            radioX = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varX, value=i,command=lambda: self.onXRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Y-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,self.dataFrame.columns.size):
            radioY = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varY, value=i,command=lambda: self.onYRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Z-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,self.dataFrame.columns.size):
            radioZ = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varZ, value=i,command=lambda: self.onZRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Color-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,self.dataFrame.columns.size):
            radioZ = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varColor, value=i,command=lambda: self.onColorRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelPointSize = Label(_labelFrame, text = 'Input Point size (eg. 0.05,0.01,1..)').grid(row=self.m,column=self.n)
        self.pointSizeField = Entry(_labelFrame)
        self.pointSizeField.grid(row=self.m,column=self.n+1)
        
        self.m+=2
        self.n=2
        plotPPTKButton = Button(_labelFrame,text= 'Plot Using PPTK', command = lambda: self.invokePPTKPlotter())
        plotPPTKButton.grid(row=self.m,column = self.n)
        
    def displayMetaData(self,_labelFrame):
        self.m =1
        self.n=1
        self.sizeLabel = Label(_labelFrame, text = ' No of Points :')
        self.sizeLabel.grid(row = self.m, column = self.n)
        self.sizeValue = Label(_labelFrame, text = str(self.pointCloud.points))
        self.sizeValue.grid(row = self.m, column = self.n+1)
        self.m += 1
        self.n =1
        self.sizeLabel = Label(_labelFrame, text = ' Format of data :')
        self.sizeLabel.grid(row = self.m, column = self.n)
        self.sizeValue = Label(_labelFrame, text = str(self.pointCloud.data))
        self.sizeValue.grid(row = self.m, column = self.n+1)

        self.m += 1
        self.n =1
        self.columnNameLabel = Label(_labelFrame, text = ' Fields in PCD File :')
        self.columnNameLabel.grid(row = self.m, column = self.n);
        self.m += 1
        self.n =2
        for attr in self.pointCloud.fields:
            self.columnNames = Label(_labelFrame, text = attr).grid(row = self.m, column = self.n)
            self.n =  self.n+1
        
        self.m += 1
        self.n =1
        self.exampleLabel = Label(_labelFrame, text = ' Example Values :')
        self.exampleLabel.grid(row = self.m, column = self.n);
        length = 10
        if(self.dataFrame.size < 10):
            length = self.dataFrame.size
        self.m += 1
        self.n =2
        col_len = len(self.pointCloud.fields)
        for i in range(0,length):
            self.n=2
            for j in range(0,col_len):
                self.exampleValue = Label(_labelFrame, text = str(self.dataFrame.values[i,j])).grid(row = self.m, column = self.n)
                self.n +=1
            self.m += 1
            
        self.loadVisualizeMenuOptions(_labelFrame)

In [4]:
class LASReader:
    def __init__(self,fileName):
        self.inFile = lp.file.File(fileName, mode = "r")
        self.data = np.vstack((self.inFile.X,self.inFile.Y,self.inFile.Z)).transpose() #x,y,z data
        self.pointFormat = self.inFile.point_format
        self.dataFrame  = pd.DataFrame(self.inFile.points)
        self.m = None
        self.n = None
        global X
        global Y
        global Z
        self.X= 0
        self.Y =1
        self.Z =2
        self.colorAttr = 0
        self.varX = IntVar()
        self.varY = IntVar()
        self.varZ = IntVar()
        self.varColor = IntVar()
        self.pointSize = None
    def onXRadioButtonSelect(self):
        self.X = self.varX.get()
        print(self.X)
        
    def onYRadioButtonSelect(self):
        self.Y = self.varY.get()
        print(self.Y)
        
    def onZRadioButtonSelect(self):
        self.Z = self.varZ.get()
    def onColorRadioButtonSelect(self):
        self.colorAttr = self.varColor.get()
        
    def invokePPTKPlotter(self):
        pptkInstance = PPTKPlotter()
        '''print(X,Y)
        data  = self.dataFrame.iloc[:,[self.X,self.Y,self.Z]]
        print(data)'''
        colorData = self.data[:,2] if self.colorAttr==0 else self.inFile.get_intensity()
        if self.pointSizeField.get() != '':
            self.pointSize = float(self.pointSizeField.get())
        else:
            self.pointSize = None
        pptkInstance.plot3D(self.data,colorData,self.pointSize)
    def loadVisualizeMenuOptions(self,_labelFrame):
        
        '''self.m+=1
        self.n=1
        labelX = Label(_labelFrame, text ='X-Axis:').grid(row=self.m,column=self.n)
        
        for i in range(0,len(self.dataFrame.values[0,0])):
            radioX = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varX, value=i,command=lambda: self.onXRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Y-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,len(self.dataFrame.values[0,0])):
            radioY = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varY, value=i,command=lambda: self.onYRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Z-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,len(self.dataFrame.values[0,0])):
            radioZ = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varZ, value=i,command=lambda: self.onZRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        '''
        self.m+=2
        self.n=1
        
        labelY = Label(_labelFrame, text ="Color-Axis:").grid(row=self.m,column=self.n)
        
        radioZ = Radiobutton(_labelFrame,text= 'Z-Axis',variable = self.varColor, value=0,command=lambda: self.onColorRadioButtonSelect()).grid(row=self.m,column=self.n+1)
        
        radioZ = Radiobutton(_labelFrame,text= 'Intensity',variable = self.varColor, value=1,command=lambda: self.onColorRadioButtonSelect()).grid(row=self.m,column=self.n+2)
        self.m+=1
        self.n=1
        labelPointSize = Label(_labelFrame, text = 'Input Point size (eg. 0.05,0.01,1..)').grid(row=self.m,column=self.n)
        self.pointSizeField = Entry(_labelFrame)
        self.pointSizeField.grid(row=self.m,column=self.n+1)
        self.m+=2
        self.n=2
        plotPPTKButton = Button(_labelFrame,text= 'Plot Using PPTK', command = lambda: self.invokePPTKPlotter())
        plotPPTKButton.grid(row=self.m,column = self.n)
        
    def displayMetaData(self,_labelFrame):
        self.m =1
        self.n=1
        self.sizeLabel = Label(_labelFrame, text = ' Shape = ')
        self.sizeLabel.grid(row = self.m, column = self.n)
        self.sizeValue = Label(_labelFrame, text = str(self.data.shape))
        self.sizeValue.grid(row = self.m, column = self.n+1)
        self.m += 1
        self.n =1
        self.columnNameLabel = Label(_labelFrame, text = ' Attributes in LAS File :')
        self.columnNameLabel.grid(row = self.m, column = self.n);
        self.m += 1
        self.n =2
        for spec in self.pointFormat:
            self.columnNames = Label(_labelFrame, text = spec.name).grid(row = self.m, column = self.n)
            self.n =  self.n+1
        
        self.m += 1
        self.n =1
        self.exampleLabel = Label(_labelFrame, text = ' Example Values :')
        self.exampleLabel.grid(row = self.m, column = self.n);
        length = 10
        if(self.dataFrame.size < 10):
            length = self.dataFrame.size
        self.m += 1
        self.n =2
        col_len = len(self.dataFrame.values[0,0])
        for i in range(0,length):
            self.n=2
            for j in range(0,col_len):
                self.exampleValue = Label(_labelFrame, text = str(self.dataFrame.values[i,0][j])).grid(row = self.m, column = self.n)
                self.n +=1
            self.m += 1
            
        self.loadVisualizeMenuOptions(_labelFrame)
    
    

In [5]:
class CSVReader:
    def __init__(self,fileName):
        self.fileName = fileName
        self.dataFrame = pd.read_csv(self.fileName)
        self.m = None
        self.n = None
        global X
        global Y
        global Z
        self.X= 0
        self.Y =1
        self.Z =2
        self.colorAttr =2
        
        self.varX = IntVar()
        self.varY = IntVar()
        self.varZ = IntVar()
        self.varColor = IntVar()
        self.pointSize = None
    def onXRadioButtonSelect(self):
        self.X = self.varX.get()
        print(self.X)
        
    def onYRadioButtonSelect(self):
        self.Y = self.varY.get()
        print(self.Y)
        
    def onZRadioButtonSelect(self):
        self.Z = self.varZ.get()
    def onColorRadioButtonSelect(self):
        self.colorAttr = self.varColor.get()    
    def invokePPTKPlotter(self):
        pptkInstance = PPTKPlotter()
        
        self.data  = self.dataFrame.iloc[:,[self.X,self.Y,self.Z]]
        if self.pointSizeField.get() != '':
            self.pointSize = float(self.pointSizeField.get())
        else:
            self.pointSize = None
        
        pptkInstance.plot3D(self.data,self.dataFrame.iloc[:,self.colorAttr],self.pointSize)
    def loadVisualizeMenuOptions(self,_labelFrame):
        
        self.m+=1
        self.n=1
        labelX = Label(_labelFrame, text ='X-Axis:').grid(row=self.m,column=self.n)
        
        for i in range(0,self.dataFrame.columns.size):
            radioX = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varX, value=i,command=lambda: self.onXRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Y-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,self.dataFrame.columns.size):
            radioY = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varY, value=i,command=lambda: self.onYRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Z-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,self.dataFrame.columns.size):
            radioZ = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varZ, value=i,command=lambda: self.onZRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelY = Label(_labelFrame, text ="Color-Axis:").grid(row=self.m,column=self.n)
        for i in range(0,self.dataFrame.columns.size):
            radioZ = Radiobutton(_labelFrame,text= 'Column '+str(i),variable = self.varColor, value=i,command=lambda: self.onColorRadioButtonSelect()).grid(row=self.m,column=self.n+1+i)
        self.m+=1
        self.n=1
        labelPointSize = Label(_labelFrame, text = 'Input Point size (eg. 0.05,0.01,1..)').grid(row=self.m,column=self.n)
        self.pointSizeField = Entry(_labelFrame)
        self.pointSizeField.grid(row=self.m,column=self.n+1)
        
        self.m+=2
        self.n=2
        plotPPTKButton = Button(_labelFrame,text= 'Plot Using PPTK', command = lambda: self.invokePPTKPlotter())
        plotPPTKButton.grid(row=self.m,column = self.n)
        
    def displayMetaData(self,_labelFrame):
        self.m =1
        self.n=1
        self.columnLabel = Label(_labelFrame, text = ' No of Columns = ')
        self.columnLabel.grid(row = self.m, column = self.n)
        self.noOfColumnLabel = Label(_labelFrame, text = str(self.dataFrame.columns.size))
        self.n +=1
        self.noOfColumnLabel.grid(row = self.m, column = self.n)
        self.m += 1
        self.n =1
        self.columnNameLabel = Label(_labelFrame, text = ' Column Names :')
        self.columnNameLabel.grid(row = self.m, column = self.n);
        self.m += 1
        self.n =1
        
        
        for i in self.dataFrame.columns:
            self.columnNames = Label(_labelFrame, text = str(i)).grid(row = self.m, column = self.n)
            self.n =  self.n+1
        self.m += 1
        self.n =1
        self.sizeLabel = Label(_labelFrame, text = ' Size = ')
        self.sizeLabel.grid(row = self.m, column = self.n)
        self.sizeValue = Label(_labelFrame, text = str(self.dataFrame.size))
        self.sizeValue.grid(row = self.m, column = self.n+1)
        self.m += 1
        self.n =1
        self.shapeLabel = Label(_labelFrame, text = ' Shape = ')
        self.shapeLabel.grid(row = self.m, column = self.n)
        self.shapeValue = Label(_labelFrame, text = str(self.dataFrame.shape))
        self.shapeValue.grid(row = self.m, column = self.n+1)
        self.m += 1
        self.n =1
        self.exampleLabel = Label(_labelFrame, text = ' Example Values :')
        self.exampleLabel.grid(row = self.m, column = self.n);
        length = 10
        if(self.dataFrame.size < 10):
            length = self.dataFrame.size
        self.m += 1
        self.n =2
        for i in range(0,length):
            self.n=2
            for j in range(0,self.dataFrame.columns.size):
                self.exampleValue = Label(_labelFrame, text = str(self.dataFrame.values[i,j])).grid(row = self.m, column = self.n)
                self.n +=1
            self.m += 1
            
        self.loadVisualizeMenuOptions(_labelFrame)
                
        
                                     
        
        
    def readCSV(self):
        self.dataframe = pd.read_csv(self.fileName)
        #print(self.dataframe)
        self.matplot = Matplotter()
        print('plot started')
        self.matplot.plotScatter3D(self.dataframe.iloc[:,1],self.dataframe.iloc[:,2],self.dataframe.iloc[:,3])
        print('plot done')

In [6]:
class Matplotter:
        
    
    def plotScatter3D(self,x,y,z):
        fig = plt.figure(figsize=(20,10))
        ax = plt.axes(projection='3d')
        print('printing')
        ax.scatter3D(x, y, z, c=z)
        fig.show()
        print('printed')
        

In [7]:
class MainPage:
    
    def __init__(self):
        self.root = Tk()
        
        self.screenWidth, self.screenHeight = self.root.winfo_screenwidth(), self.root.winfo_screenheight()
        print(self.screenHeight,self.screenWidth)
        self.root.state('zoomed')
        self.fileNameLabel = None
        self.fileType = None
        self.visualizationFrame =None
        #self.root.attributes('-fullscreen', True)
        #self.myLabel = None
    def streamFromAPI(self):
        pptkInstance = PPTKPlotter()
        self.API = self.apiField.get()
        pptkInstance.plotStream3D(self.API)
    def processFile(self):
        self.loadVisualizationArea()
        if self.fileType == 'csv':
            csvReader  = CSVReader(self.fileName.name)
            csvReader.displayMetaData(self.visualizationFrame)
        if self.fileType == 'las':
            lasReader = LASReader(self.fileName.name)
            lasReader.displayMetaData(self.visualizationFrame)
        if self.fileType == 'pcd':
            pcdReader = PCDReader(self.fileName.name)
            pcdReader.displayMetaData(self.visualizationFrame)
        
    
    
    def inputFile(self):
        self.fileName = askopenfile(mode ='r') #restrict file types
        if self.fileNameLabel is not None:
            self.fileNameLabel.destroy()
        self.fileNameLabel = Label(self.headerFrame, text=self.fileName.name)
        self.fileNameLabel.grid(row=2, column=0)
        self.fileType = self.fileName.name.split('.')[-1]
        self.fileTypeLabel = Label(self.headerFrame, text =self.fileType)
        self.fileTypeLabel.grid(row=3,column=0)
        self.fileProcessButton = Button(self.headerFrame, text='Process File',command= lambda: self.processFile())
        self.fileProcessButton.grid(row=3,column=2)
        
    def loadStreamingInputArea(self):
        self.apiFieldLabel = Label(self.headerFrame,text = '  Input API for Streaming : ')
        self.apiFieldLabel.grid(row=0,column=3)
        self.apiField = Entry(self.headerFrame,width= 50)
        self.apiField.grid(row=0,column=4)
        self.streamButton  = Button(self.headerFrame, text  = 'Stream' , command= lambda: self.streamFromAPI())
        self.streamButton.grid(row=0,column =5)
        
    def loadFileInputArea(self):
        self.inputFileLabelText = Label(self.headerFrame, text= ' Upload Your File : ')
        self.inputFileLabelText.grid(row=0,column=0)
        self.inputFileButton = Button(self.headerFrame,text= 'Click to Upload', command=lambda: self.inputFile())
        self.inputFileButton.grid(row=0,column=2)
    
    def loadHeader(self):
        self.root.title('Point Cloud Visualization Tool')
        self.headerFrame = LabelFrame(self.root,text='Point Cloud Visualization',padx=10)
        #self.headerFrame.pack(expand='yes',fill=X,side=TOP)
        self.headerFrame.place(x=0,y=0,height=int(self.screenHeight*0.16),width=int(self.screenWidth*0.95))
        self.loadFileInputArea()
        self.loadStreamingInputArea()
    def loadVisualizationArea(self):
        if self.visualizationFrame is not None:
            self.visualizationFrame.destroy()
        self.visualizationFrame = LabelFrame(self.root,text="Visualization Area")
        #self.visualizationFrame.pack(expand='yes',fill='both')
        self.visualizationFrame.place(x=0,y=int(self.screenHeight*0.16),height=int(self.screenHeight*0.8),width=int(self.screenWidth*0.95))
    def loadPage(self):
        self.loadHeader()
        self.loadVisualizationArea()
        self.root.mainloop()
        

        

In [8]:
app = MainPage()
app.loadPage()

768 1366
